# IMPORT

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sqlite3 as sql
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import SoccerNet
from SoccerNet.Downloader import SoccerNetDownloader
import copy

# Database connection

In [25]:
db_path = "/scratch/users/mdelabrassinne/Database/SoccerDB.db"
con = sql.connect(db_path)
curs = con.cursor()
curs.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(curs.fetchall())

[('VIDEO',), ('Sequence',), ('IMAGE',)]
('CREATE TABLE VIDEO(Path_video TEXT PRIMARY KEY, fps INTEGER, quality INTEGER, half INTEGER, training_stage TEXT, duration INTEGER, date TEXT)',)


# Get training matches

In [26]:
curs.execute("SELECT Path_video FROM VIDEO WHERE training_stage = 'Training'")
train_matches = copy.deepcopy(list(zip(*curs.fetchall()))[0])
print(train_matches)

('/scratch/users/mdelabrassinne/Database/europe_uefa-champions-league/2015-2016/2015-11-03_-_22-45_Real_Madrid_1_-_0_Paris_SG/2_224p.mkv', '/scratch/users/mdelabrassinne/Database/france_ligue-1/2016-2017/2017-04-18_-_19-30_Metz_2_-_3_Paris_SG/2_224p.mkv', '/scratch/users/mdelabrassinne/Database/england_epl/2016-2017/2016-11-19_-_18-00_Southampton_0_-_0_Liverpool/1_224p.mkv', '/scratch/users/mdelabrassinne/Database/england_epl/2015-2016/2016-02-27_-_18-00_Southampton_1_-_2_Chelsea/1_224p.mkv', '/scratch/users/mdelabrassinne/Database/england_epl/2015-2016/2015-08-08_-_19-30_Chelsea_2_-_2_Swansea/1_224p.mkv', '/scratch/users/mdelabrassinne/Database/england_epl/2016-2017/2016-12-14_-_22-45_Middlesbrough_0_-_3_Liverpool/2_224p.mkv', '/scratch/users/mdelabrassinne/Database/germany_bundesliga/2016-2017/2016-12-20_-_18-00_Bayern_3_-_0_Leipzig/2_224p.mkv', '/scratch/users/mdelabrassinne/Database/spain_laliga/2016-2017/2017-05-14_-_21-00_Las_Palmas_1_-_4_Barcelona/1_224p.mkv', '/scratch/users/md

# From db path to features path

In [27]:
def get_feature_paths(match_list):
    
    features_list = []
    for match in match_list:
        if match[-10] == '1':
            end_path = "1_baidu_soccer_embeddings.npy"
        else:
            end_path = "2_baidu_soccer_embeddings.npy"
            
        direc = os.path.dirname(match) ## dir of file
        ## once you're at the directory level you want, with the desired directory as the final path node:
        dirname1 = os.path.split(direc)[0]
        dirname2 = os.path.split(direc)[1]
        good_dir = dirname2.replace("_", " ")
        features_list.append(os.path.join(dirname1,good_dir,end_path))
    return features_list

features = get_feature_paths(train_matches)
print(features)


['/scratch/users/mdelabrassinne/Database/europe_uefa-champions-league/2015-2016/2015-11-03 - 22-45 Real Madrid 1 - 0 Paris SG/2_baidu_soccer_embeddings.npy', '/scratch/users/mdelabrassinne/Database/france_ligue-1/2016-2017/2017-04-18 - 19-30 Metz 2 - 3 Paris SG/2_baidu_soccer_embeddings.npy', '/scratch/users/mdelabrassinne/Database/england_epl/2016-2017/2016-11-19 - 18-00 Southampton 0 - 0 Liverpool/1_baidu_soccer_embeddings.npy', '/scratch/users/mdelabrassinne/Database/england_epl/2015-2016/2016-02-27 - 18-00 Southampton 1 - 2 Chelsea/1_baidu_soccer_embeddings.npy', '/scratch/users/mdelabrassinne/Database/england_epl/2015-2016/2015-08-08 - 19-30 Chelsea 2 - 2 Swansea/1_baidu_soccer_embeddings.npy', '/scratch/users/mdelabrassinne/Database/england_epl/2016-2017/2016-12-14 - 22-45 Middlesbrough 0 - 3 Liverpool/2_baidu_soccer_embeddings.npy', '/scratch/users/mdelabrassinne/Database/germany_bundesliga/2016-2017/2016-12-20 - 18-00 Bayern 3 - 0 Leipzig/2_baidu_soccer_embeddings.npy', '/scrat

# Model Creation